In [1]:
from video_llama.common.config import Config
from video_llama.common.dist_utils import get_rank
from video_llama.common.registry import registry
from video_llama.conversation.conversation_video import Chat, Conversation, default_conversation,SeparatorStyle,conv_llava_llama_2

#%%
# imports modules for registration
from video_llama.datasets.builders import *
from video_llama.models import *
from video_llama.processors import *
from video_llama.runners import *
from video_llama.tasks import *

import os

os.chdir('/srv/flash1/pputta7/projects/lm-nav')

[2023-10-24 21:28:32,437] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/coc/testnvme/pputta7/mambaforge/envs/lmnav/lib/python3.9/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/coc/testnvme/pputta7/mambaforge/envs/lmnav/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/coc/testnvme/pputta7/mambaforge/envs/lmnav/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvisio

In [2]:
class Args:
    def __init__(self):
        self.cfg_path = "Video-LLaMA/eval_configs/video_llama_eval_only_vl.yaml"
        self.model_type = "llama_v2"
        self.gpu_id = 0
        self.options = []

args = Args()
cfg = Config(args)

In [3]:
model_config = cfg.model_cfg
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))
model.eval()
vis_processor_cfg = cfg.datasets_cfg.webvid.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)
chat = Chat(model, vis_processor, device='cuda:{}'.format(args.gpu_id))
print('Initialization Finished')

Loading VIT
Loading VIT Done
Loading Q-Former


Using pad_token, but it is not set yet.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Load first Checkpoint: /srv/flash1/pputta7/data/models/Video-LLaMA-2-13B-Finetuned/VL_LLaMA_2_13B_Finetuned.pth
Initialization Finished


In [4]:
chat_state = conv_llava_llama_2.copy()
chat_state.system =  "You are a navigational robot with egocentric vision that navigates from a start to goal point."
img_list = []
llm_message = chat.upload_video_without_audio("/srv/flash1/pputta7/projects/lm-nav/experiments/lmnav-1env/bc/lora+clip+max_trajectory_length=200+batch_size=9/eval/ckpt.56.pth/videos/episode=17-ckpt=56-.mp4", chat_state, img_list)

/srv/flash1/pputta7/projects/lm-nav/experiments/lmnav-1env/bc/lora+clip+max_trajectory_length=200+batch_size=9/eval/ckpt.56.pth/videos/episode=17-ckpt=56-.mp4


In [5]:
user_message = "Describe what happened during the route in detail."
chat.ask(user_message, chat_state)
llm_message = chat.answer(conv=chat_state,
                              img_list=img_list,
                              num_beams=1,
                              temperature=0.6,
                              max_new_tokens=600,
                              max_length=2000)[0]
print(chat_state.get_prompt())

[INST] <<SYS>>
You are a navigational robot with egocentric vision that navigates from a start to goal point.
<</SYS>>

<Video><ImageHere></Video> The video contains 8 frames sampled at 0.0, 3.9, 7.8, 11.8, 15.7, 19.6, 23.6, 27.5 seconds.  Describe what happened during the route in detail. [/INST] First, the video shows a room with a green and white wall, a yellow and white couch, and a white and brown chair. Next, the video shows a living room with a yellow and white couch, a white and brown chair, and a white and brown couch. Then, the video shows a room with a yellow and white couch, a white and brown couch, and a white and brown chair. After that, the video shows a living room with a yellow and white couch, a white and brown couch, and a white and brown chair. The video then shows a room with a yellow and white couch, a white and brown couch, and a white and brown chair. Next, the video shows a room with a yellow and white couch, a white and brown couch, and a white and brown chair